In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
dataset = pd.read_csv('/content/housing.csv')
dataset = dataset.dropna()
dataset.head()
Y = dataset['median_house_value']
X = dataset.loc[:,'longitude':'median_income']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()
x_test_np = x_test.to_numpy()
y_test_np = y_test.to_numpy()
import torch
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

In [15]:
print(x_train_np.shape)

(12790, 8)


In [16]:
print(y_train_np.shape)

(12790,)


In [0]:
def model_loss(model, dataset, train = False, optimizer = None):
  performance = L1Loss()
  score_metric = R2Score()
  avg_loss = 0
  avg_score = 0
  count = 0
  for input, output in iter(dataset):
    predictions = model.feed(input)
    loss = performance(predictions, output)
    score_metric.update([predictions, output])
    score = score_metric.compute()
    if(train):
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    avg_loss += loss.item()
    avg_score += score
    count += 1
  return avg_loss / count, avg_score / count

In [0]:
class CnnRegressor(torch.nn.Module):
  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs
    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer = MaxPool1d(1)
    self.conv_layer = Conv1d(batch_size, 128, 1)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(128, 64)
    self.output_layer = Linear(64, outputs)
  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pooling_layer(output)
    output = relu(self.conv_layer(output))
    output = self.flatten_layer(output)
    output = self.linear_layer(output)
    output = self.output_layer(output)
    return output

In [49]:

batch_size = 64
model = CnnRegressor(batch_size, X.shape[1], 1)
model.cuda()


CnnRegressor(
  (input_layer): Conv1d(8, 64, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [64]:
epochs = 150
optimizer = SGD(model.parameters(), lr=1e-5)
optimizer = Adam(model.parameters())
inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=False, drop_last=True)
for epoch in range(epochs):
  avg_loss, avg_r2_score = model_loss(model, loader, train=True, optimizer=optimizer)
  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))


Epoch 1:
	Loss = 39780.575612437184
	R^2 Score = 0.7297673636081922
Epoch 2:
	Loss = 39884.88602190641
	R^2 Score = 0.7350717083122654
Epoch 3:
	Loss = 39467.67456422739
	R^2 Score = 0.7372940679145172
Epoch 4:
	Loss = 39572.763789651384
	R^2 Score = 0.7367893929620674
Epoch 5:
	Loss = 39469.31171482412
	R^2 Score = 0.736317678475729
Epoch 6:
	Loss = 39624.86306532663
	R^2 Score = 0.7356826838984497
Epoch 7:
	Loss = 39744.38973186244
	R^2 Score = 0.7334547400423774
Epoch 8:
	Loss = 39627.9980174309
	R^2 Score = 0.7355531516990677
Epoch 9:
	Loss = 39626.21399968593
	R^2 Score = 0.7349430114893877
Epoch 10:
	Loss = 39481.9446549152
	R^2 Score = 0.7353004381165841
Epoch 11:
	Loss = 39702.17377905151
	R^2 Score = 0.7356395905400227
Epoch 12:
	Loss = 39802.27639172425
	R^2 Score = 0.7325753568342948
Epoch 13:
	Loss = 40493.78758048053
	R^2 Score = 0.7289737568883916
Epoch 14:
	Loss = 39903.33032349246
	R^2 Score = 0.7337048364257535
Epoch 15:
	Loss = 39511.07285450691
	R^2 Score = 0.7335688

In [65]:
  inputs = torch.from_numpy(x_test_np).cuda().float()
  outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()
  tensor = TensorDataset(inputs, outputs)
  loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
  avg_loss, avg_r2_score = model_loss(model, loader)
  print("The model's L1 loss is: " + str(avg_loss))
  print("The model's R^2 score is: " + str(avg_r2_score))

The model's L1 loss is: 43273.36985809949
The model's R^2 score is: 0.7016275815828352
